In [ ]:
!pip install pyspark==3.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212.4 MB 71 kB/s 
     |████████████████████████████████| 198 kB 47.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=b46ceefb1c0820f2eca7b7314becdb41febded08f11265c2789900825708be2c
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import *
from pyspark.sql.types import *


spark = SparkSession.builder.master("local[8]").config("spark.driver.memory", "8g").config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2").appName("aig").enableHiveSupport().getOrCreate()
sql = spark.sql

In [ ]:
logs = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "sun.ru77.ru:39092").option("startingOffsets", "latest").option('subscribe', 'logs').load()

In [ ]:
lines = logs.select(logs.value.alias("line"))

In [ ]:
lines

DataFrame[line: binary]

In [ ]:
ips = lines.select(
    regexp_extract("line", r'^([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)', 1).alias('ip'),
    regexp_extract("line", r'^[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+ \- \- \[([^\]]+)\]', 1).alias('ts'),
    current_timestamp().alias('current_ts')
).withColumn('ts', to_timestamp(col("ts"), 'dd/MMM/yyyy:HH:mm:ss Z'))

In [ ]:
ips

DataFrame[ip: string, ts: timestamp, current_ts: timestamp]

In [ ]:
ipCounts = ips.groupBy(ips.ip).count()

In [ ]:
query = ipCounts.writeStream.outputMode("complete").queryName("logs_ips_1").format("memory").start()

In [ ]:
query.status

{'isDataAvailable': False,
 'isTriggerActive': True,
 'message': 'Getting offsets from KafkaV2[Subscribe[logs]]'}

In [ ]:
spark.sql("SELECT * FROM logs_ips_1").show(truncate=False)

+---+-----+
|ip |count|
+---+-----+
+---+-----+



In [ ]:
query.recentProgress

[]

In [ ]:
query.explain()

No physical plan. Waiting for data.


In [ ]:
ipCounts = ips.groupBy(window(ips.ts, "30 seconds", "15 seconds"), ips.ip).agg(count('ip').alias('cnt'), max('current_ts').alias('last_ts'))

In [ ]:
query2 = ipCounts.writeStream.outputMode("complete").queryName("logs_ips_2").format("memory").start()

In [ ]:
spark.sql("SELECT * FROM logs_ips_2").show(truncate=False)

+------+---+---+-------+
|window|ip |cnt|last_ts|
+------+---+---+-------+
+------+---+---+-------+



In [ ]:
query2.stop()

In [ ]:
ipCounts = ips.groupBy(window(ips.ts, "1 minutes", "1 minutes"), ips.ip).agg(count('ip').alias('cnt'), max('current_ts').alias('last_ts'))

In [ ]:
query = ipCounts.withWatermark('last_ts', '30 seconds').writeStream.outputMode("update").queryName("logs_ips").format("memory").start()

In [ ]:
spark.sql("SELECT * FROM logs_ips ORDER BY window").show(truncate=False)